In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Classroom/DSF/DSF Final Proj/DSF Project Fall 2021/data/2021-22/gws

/content/drive/.shortcut-targets-by-id/1uITAlwG_KLn52Rc6b51jPmC6XmnWK3r-/DSF Project Fall 2021/data/2021-22/gws


In [ ]:
ls

gw1.csv   gw2.gsheet  gw5.csv  gw8.csv           xP1.csv  xP4.csv  xP7.csv
gw23.csv  gw3.csv     gw6.csv  merged_gw.csv     xP2.csv  xP5.csv  xP8.csv
gw2.csv   gw4.csv     gw7.csv  merged_gw.gsheet  xP3.csv  xP6.csv


In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import permutation_test_score as perm_test
from sklearn.metrics import mean_squared_error
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import ttest_ind
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median', add_indicator=True)
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDClassifier
import requests
from IPython.display import display, Markdown, Latex

In [ ]:
main_data = pd.read_csv("gw23.csv")
data = main_data.copy()
players =data['name'].unique()

print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
data.isna().sum()

name                 0
position             0
team                 0
xP                   0
assists              0
bonus                0
bps                  0
clean_sheets         0
creativity           0
element              0
fixture              0
goals_conceded       0
goals_scored         0
ict_index            0
influence            0
kickoff_time         0
minutes              0
opponent_team        0
own_goals            0
penalties_missed     0
penalties_saved      0
red_cards            0
round                0
saves                0
selected             0
team_a_score         0
team_h_score         0
threat               0
total_points         0
transfers_balance    0
transfers_in         0
transfers_out        0
value                0
was_home             0
yellow_cards         0
dtype: int64

In [ ]:
data.drop(columns='team_h_score', inplace=True)

In [ ]:
data

,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,threat,total_points,transfers_in,transfers_out,value,was_home,yellow_cards
0,DEF,Man Utd,1.0,0,0,6,0,0.0,286,228,0,0,0.0,0.0,0,19,0,0,0,0,0,11164,0.0,1,42,368,49,True,0
1,FWD,Aston Villa,1.0,0,0,6,0,0.0,49,225,0,0,0.0,0.0,0,8,0,0,0,0,0,178454,0.0,1,0,15944,45,False,0
2,MID,Watford,1.0,0,0,6,0,0.0,394,230,0,0,0.0,0.0,0,15,0,0,0,0,0,901,0.0,1,0,8,45,True,0
3,MID,Watford,1.0,0,0,6,0,0.0,394,164,0,0,0.0,0.0,0,5,0,0,0,0,0,901,0.0,1,0,8,45,False,0
4,MID,Southampton,5.0,0,0,22,0,28.8,341,229,1,0,5.1,22.2,90,12,0,0,0,0,0,300618,0.0,3,36877,21374,63,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,MID,Leicester,1.0,0,0,6,0,0.0,216,227,0,0,0.0,0.0,0,4,0,0,0,0,0,74867,0.0,1,22,2195,48,True,0
769,DEF,Newcastle,0.6,0,0,6,0,0.0,292,226,0,0,0.0,0.0,0,10,0,0,0,0,0,16202,0.0,1,173,765,49,False,0
770,MID,Southampton,3.7,1,0,17,0,11.0,336,229,0,0,2.8,17.2,45,12,0,0,0,0,0,16138,0.0,5,305,968,59,True,0
771,GK,Brighton,1.0,0,0,6,0,0.0,65,227,0,0,0.0,0.0,0,9,0,0,0,0,0,618,0.0,1,0,2,45,False,0


In [ ]:
data.xP = data.xP+1

In [ ]:
data.total_points = data.total_points+1

In [ ]:
data.total_points.min()

0

In [ ]:
data.to_csv('vis_prj1.csv',index=False)

In [ ]:
def damp(grouped_df,col,damping=0.8):
  new_df = pd.DataFrame(data = {'name':grouped_df['name'].unique()}) 
  for player in new_df['name'].unique():
    val = 0;
    for season in ['2016-17','2017-18','2018-19','2019-20','2020-21']:
      df = grouped_df[((grouped_df.name == player) & (grouped_df.season_x == season))]
      
      if season == '2016-17':
        if len(df[col].values) == 0:
          val =   0
        else:
          val = df[col].values[0]
        continue;
      
      if len(df[col].values) == 0:
        val = (val * damping + 0) / (1+damping)
      else:
        val = (val * damping + df[col].values[0]) / (1+damping)
        # if col == 'goals_conceded' and player == 'Hamza Choudhury':
        #   print(val)
    new_df.loc[new_df.name == player,[col]] = val


  return new_df

In [ ]:
data['minutes'].value_counts()

0     32231
90    23026
45      876
1       516
77      251
      ...  
43       13
49       12
40       11
44       10
46        8
Name: minutes, Length: 91, dtype: int64

In [ ]:
# data = data[data['name'] == 'Hamza Choudhury']
data = data[(data['minutes']>0)]
final_data = pd.DataFrame(data = {'name':data['name'].unique()}) 


for i in ['assists','goals_scored','goals_conceded','ict_index','influence','creativity', 'threat','value']:
  temp_df_mean = data.groupby(['name','season_x']).mean().reset_index()
  val = damp(temp_df_mean,i)
  for player in data['name'].unique():

    final_data.loc[final_data.name == player,[i]] = val[val.name == player][i].values[0]

temp_df_sum = data.groupby(['name','season_x']).sum().reset_index()
temp_df_sum['minutes'] = temp_df_sum['minutes']/38 
final_data['minutes'] =  damp(temp_df_sum,'minutes')['minutes']
 
# final_data
# final_data[final_data['name'] == 'Hamza Choudhury']


0.018442310623380583
0.012345679012345682
0.36290199664685263
1.1942234415485442
5.954366712391403
2.6359929888736473
3.3552049992379214
43.96349641822893
12.478100257502467


In [ ]:
final_data[(final_data['name'] == 'Hamza Choudhury')]

,name,assists,goals_scored,goals_conceded,ict_index,influence,creativity,threat,value,minutes
236,Hamza Choudhury,0.018442,0.012346,0.362902,1.194223,5.954367,2.635993,3.355205,43.963496,5.820879


In [ ]:
x= data[(data['minutes']>0)]
y =x.groupby(['name','season_x']).mean().reset_index()
y[(y['name'] == 'Hamza Choudhury')]

,name,season_x,Unnamed: 0,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
587,Hamza Choudhury,2017-18,13750.500000,0.125,0.0,7.500000,0.000000,2.525000,571.0,317.00,1.125000,0.00,1.300000,8.500000,46.875000,11.500000,0.0,0.0,0.0,0.00,32.875000,0.0,453.750000,1.375000,2.25,2.000000,1.625000,64.000000,82.750000,18.750000,44.125,0.625000,0.125,32.875000
588,Hamza Choudhury,2018-19,25286.222222,0.000,0.0,8.888889,0.444444,8.188889,230.0,271.00,0.444444,0.00,2.733333,13.288889,65.888889,9.444444,0.0,0.0,0.0,0.00,27.555556,0.0,7800.666667,1.111111,1.00,5.666667,2.222222,191.777778,584.555556,392.777778,43.000,0.444444,0.000,27.555556
589,Hamza Choudhury,2019-20,35669.900000,0.050,0.0,7.650000,0.150000,3.500000,180.0,170.45,0.600000,0.05,1.655000,8.880000,44.150000,13.200000,0.0,0.0,0.0,0.05,19.250000,0.0,62632.200000,1.100000,1.40,4.150000,1.750000,1265.500000,5668.200000,4402.700000,43.800,0.550000,0.100,19.250000
590,Hamza Choudhury,2020-21,57005.400000,0.000,0.0,5.900000,0.200000,1.350000,234.0,203.40,0.200000,0.00,0.760000,3.400000,31.100000,8.800000,0.0,0.0,0.0,0.00,20.700000,0.0,4256.000000,1.500000,0.80,2.900000,1.400000,-101.800000,97.500000,199.300000,47.300,0.300000,0.100,20.700000


In [ ]:
x= data[(data['minutes']>0)]
y =x.groupby(['name','season_x']).sum().reset_index()
y.sort_values(['goals_scored'],ascending=False)

,name,season_x,Unnamed: 0,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
1159,Mohamed Salah,2017-18,434637,12,26,881,15,942.5,8424,6698,29,32,454.4,1496.2,2905,389,0,1,0,0,686,0,96983380,47.0,72.0,2109.0,303,2342721,4177480,1834759,3562,19,1,686
595,Harry Kane,2017-18,449951,2,31,781,15,502.1,14578,7203,34,29,407.4,1214.4,3074,374,0,1,0,0,737,0,77531321,50.0,59.0,2355.0,217,2860192,5902449,3042257,4736,19,5,737
594,Harry Kane,2016-17,96514,7,33,920,14,548.4,12090,6017,19,29,336.0,1279.0,2523,303,0,1,0,0,620,0,20597063,38.0,56.0,1530.0,224,1120918,2379855,1258937,3356,16,3,620
740,Jamie Vardy,2019-20,1273711,7,34,775,11,418.4,5810,6543,38,23,288.8,1026.0,3032,356,0,1,0,0,753,0,74662264,44.0,57.0,1455.0,210,752945,4940635,4187690,3346,19,3,753
598,Harry Kane,2020-21,1961001,14,40,880,12,659.1,13580,6522,39,23,355.9,1318.2,3083,362,0,0,0,0,675,0,103084622,51.0,57.0,1585.0,242,5244663,8551448,3306785,3898,17,1,675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,John Egan,2020-21,1721312,1,4,410,4,50.9,10943,5660,48,0,74.5,544.6,2628,321,0,0,0,1,583,0,7382643,28.0,40.0,150.0,57,-498911,40008,538919,1477,15,7,583
813,John Fleck,2020-21,1778695,3,1,353,5,530.8,10726,6554,47,0,103.7,351.0,2573,318,0,0,0,0,670,0,356151,29.0,38.0,154.0,70,-10279,15305,25584,1746,15,4,670
815,John Lundstram,2020-21,1533958,2,2,244,3,346.9,9940,4754,41,0,79.3,245.4,2027,282,0,1,0,1,488,0,6242891,30.0,33.0,206.0,48,-649495,46235,695730,1414,16,8,488
818,John Ruddy,2018-19,25737,0,0,25,1,0.0,421,290,0,0,2.7,26.6,90,5,0,0,0,0,29,3,15330,0.0,2.0,0.0,7,796,959,163,42,1,0,29


In [ ]:
x= data[(data['minutes']>0)]
y =x.groupby(['name','season_x']).count().reset_index()
y.sort_values(['goals_scored'],ascending=False)

,name,season_x,Unnamed: 0,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
966,Leander Dendoncker,2019-20,38,38,0,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38
333,César Azpilicueta,2016-17,38,38,0,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38
842,Jonny Evans,2019-20,38,38,0,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38
825,Jonas Lössl,2017-18,38,38,0,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38
107,Andros Townsend,2018-19,38,38,0,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,Martin Kelly,2020-21,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
344,Dane Scarlett,2020-21,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
591,Hannibal Mejbri,2020-21,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1476,Sergio Romero,2017-18,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
final_data
x= data[(data['minutes']>0)]
y =x.groupby(['name','season_x']).mean().reset_index()
y.sort_values(['goals_scored'],ascending=False)

,name,season_x,Unnamed: 0,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
184,Ben Gibson,2018-19,22429.000000,0.000000,0.000000,17.000000,0.000000,0.400000,491.0,182.000000,3.000000,1.000000,7.600000,42.400000,62.000000,8.000000,0.0,0.000000,0.0,0.000000,19.000000,0.000000,5.680000e+03,5.000000,1.000000,33.000000,6.000000,-32.000000,51.000000,83.000000,48.000000,1.000000,1.000000,19.000000
594,Harry Kane,2016-17,3217.133333,0.233333,1.100000,30.666667,0.466667,18.280000,403.0,200.566667,0.633333,0.966667,11.200000,42.633333,84.100000,10.100000,0.0,0.033333,0.0,0.000000,20.666667,0.000000,6.865688e+05,1.266667,1.866667,51.000000,7.466667,37363.933333,79328.500000,41964.566667,111.866667,0.533333,0.100000,20.666667
1159,Mohamed Salah,2017-18,12073.250000,0.333333,0.722222,24.472222,0.416667,26.180556,234.0,186.055556,0.805556,0.888889,12.622222,41.561111,80.694444,10.805556,0.0,0.027778,0.0,0.000000,19.055556,0.000000,2.693983e+06,1.305556,2.000000,58.583333,8.416667,65075.583333,116041.111111,50965.527778,98.944444,0.527778,0.027778,19.055556
1470,Sergio Agüero,2017-18,11554.280000,0.240000,0.880000,29.600000,0.520000,22.832000,257.0,161.360000,0.480000,0.840000,12.100000,38.656000,78.400000,9.520000,0.0,0.000000,0.0,0.000000,16.600000,0.000000,1.063703e+06,1.400000,2.120000,59.360000,6.760000,45273.040000,120546.600000,75273.560000,116.640000,0.600000,0.080000,16.600000
595,Harry Kane,2017-18,12160.837838,0.054054,0.837838,21.108108,0.405405,13.570270,394.0,194.675676,0.918919,0.783784,11.010811,32.821622,83.081081,10.108108,0.0,0.027027,0.0,0.000000,19.918919,0.000000,2.095441e+06,1.351351,1.594595,63.648649,5.864865,77302.486486,159525.648649,82223.162162,128.000000,0.513514,0.135135,19.918919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,John Egan,2020-21,55526.193548,0.032258,0.129032,13.225806,0.129032,1.641935,353.0,182.580645,1.548387,0.000000,2.403226,17.567742,84.774194,10.354839,0.0,0.000000,0.0,0.032258,18.806452,0.000000,2.381498e+05,0.903226,1.290323,4.838710,1.838710,-16093.903226,1290.580645,17384.483871,47.645161,0.483871,0.225806,18.806452
813,John Fleck,2020-21,57377.258065,0.096774,0.032258,11.387097,0.161290,17.122581,346.0,211.419355,1.516129,0.000000,3.345161,11.322581,83.000000,10.258065,0.0,0.000000,0.0,0.000000,21.612903,0.000000,1.148874e+04,0.935484,1.225806,4.967742,2.258065,-331.580645,493.709677,825.290323,56.322581,0.483871,0.129032,21.612903
815,John Lundstram,2020-21,54784.214286,0.071429,0.071429,8.714286,0.107143,12.389286,355.0,169.785714,1.464286,0.000000,2.832143,8.764286,72.392857,10.071429,0.0,0.035714,0.0,0.035714,17.428571,0.000000,2.229604e+05,1.071429,1.178571,7.357143,1.714286,-23196.250000,1651.250000,24847.500000,50.500000,0.571429,0.285714,17.428571
818,John Ruddy,2018-19,25737.000000,0.000000,0.000000,25.000000,1.000000,0.000000,421.0,290.000000,0.000000,0.000000,2.700000,26.600000,90.000000,5.000000,0.0,0.000000,0.0,0.000000,29.000000,3.000000,1.533000e+04,0.000000,2.000000,0.000000,7.000000,796.000000,959.000000,163.000000,42.000000,1.000000,0.000000,29.000000


In [ ]:
final_data[final_data['name'] == 'Hamza Choudhury']

,name,assists,goals_scored,goals_conceded,ict_index,influence,creativity,threat,value,minutes
236,Hamza Choudhury,0.018442,0.012346,0.362902,1.194223,5.954367,2.635993,3.355205,43.963496,5.820879


In [ ]:
# bonus

for player in data['name'].unique():
  value = data[data['name'] == player]['bonus'].value_counts()
  player_mapping = {}
  for i in [0,1,2,3]:
    player_mapping[i] = 0
  for i in value.index.tolist():
    player_mapping[i] = value[i] / sum(value.to_list())

  p_arr = []
  for i in [0,1,2,3]:
    p_arr.append(player_mapping[i])
  # print(sum(p_arr))
  final_data.loc[final_data.name == player,['bonus']] = np.random.choice(np.arange(0, 4), p=p_arr)


# Kenny Tete
# x = data[data['name'] == "Kenny Tete"]['bonus'].value_counts()
# x[2]

['Adam Forshaw' 'Adam Lallana' 'Adrián San Miguel del Castillo'
 'Alex Iwobi' 'Alex Oxlade-Chamberlain' 'Andrew Robertson'
 'Andros Townsend' 'Andy Carroll' 'Anthony Martial' 'Arthur Masuaku'
 'Bamidele Alli' 'Ben Gibson' 'Ben Mee' 'Branislav Ivanovic'
 'Callum Wilson' 'Calum Chambers' 'Charlie Austin' 'Cheikhou Kouyaté'
 'Christian Fuchs' 'Connor Wickham' 'Craig Dawson' 'Cédric Soares'
 'César Azpilicueta' 'Daniel Amartey' 'Danny Rose' 'David de Gea'
 'Demarai Gray' 'Divock Origi' 'Eric Bailly' 'Eric Dier' 'Erik Lamela'
 'Erik Pieters' 'Fabian Delph' 'Federico Fernández' 'Fernando Luiz Rosa'
 'Fraser Forster' 'Gary Cahill' 'Georginio Wijnaldum' 'Granit Xhaka'
 'Gylfi Sigurdsson' 'Harry Kane' 'Hugo Lloris' 'Héctor Bellerín'
 'Jack Cork' 'Jake Livermore' 'James McCarthy' 'James Ward-Prowse'
 'Jamie Vardy' 'Joel Ward' 'Johann Berg Gudmundsson' 'John Stones'
 'Jonny Evans' 'Jordan Henderson' 'Joshua King' 'Juan Mata'
 'Kasper Schmeichel' 'Kelechi Iheanacho' 'Kevin De Bruyne' 'Kyle Walker'

In [ ]:
# yellow_cards
for player in data['name'].unique():
  value = data[data['name'] == player]['yellow_cards'].value_counts()
  player_mapping = {}
  for i in [0,1]:
    player_mapping[i] = 0
  for i in value.index.tolist():
    player_mapping[i] = value[i] / sum(value.to_list())

  p_arr = []
  for i in [0,1]:
    p_arr.append(player_mapping[i])

  final_data.loc[final_data.name == player,['bonus']] = np.random.choice(np.arange(0, 2), p=p_arr)
final_data['bonus'].value_counts()

0.0    537
1.0     64
Name: bonus, dtype: int64

In [ ]:
# clean sheets
for player in data['name'].unique():
  value = data[data['name'] == player]['clean_sheets'].value_counts()
  player_mapping = {}
  for i in [0,1]:
    player_mapping[i] = 0
  for i in value.index.tolist():
    player_mapping[i] = value[i] / sum(value.to_list())

  p_arr = []
  for i in [0,1]:
    p_arr.append(player_mapping[i])

  final_data.loc[final_data.name == player,['clean_sheets']] = np.random.choice(np.arange(0, 2), p=p_arr)
final_data['clean_sheets'].value_counts()

0.0    468
1.0    133
Name: clean_sheets, dtype: int64

In [ ]:
# final_data['bfs'] = data.groupby("name").median().reset_index()['bfs']
x = data.groupby("name").median().reset_index()
final_data['bps'] = x['bps']
final_data

,name,assists,goals_scored,goals_conceded,ict_index,influence,creativity,threat,value,minutes,bonus,clean_sheets,bps
0,Adam Forshaw,0.002295,0.000000,0.041314,0.117629,0.316279,0.587113,0.275424,1.755830,19.569526,0.0,1.0,3.0
1,Adam Lallana,0.052232,0.045049,0.485304,2.770911,8.199072,10.226071,9.323228,62.916016,74.761028,0.0,1.0,14.0
2,Adrián San Miguel del Castillo,0.000000,0.000000,2.026543,1.601475,16.090987,0.024387,0.000000,39.384984,24.161192,0.0,0.0,12.0
3,Alex Iwobi,0.097522,0.047931,0.864306,3.409832,10.063514,12.346996,11.669281,57.633463,71.637427,0.0,0.0,17.0
4,Alex Oxlade-Chamberlain,0.070138,0.082528,0.423794,2.189492,7.403909,6.023490,8.538180,62.420209,74.540226,0.0,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,Jaden Philogene-Bidace,0.000000,0.000000,0.000000,0.055556,0.000000,0.277778,0.000000,24.444444,70.546531,0.0,0.0,13.0
597,Carney Chukwuemeka,0.000000,0.000000,0.000000,0.166667,0.277778,0.305556,1.388889,25.000000,58.732943,0.0,0.0,9.5
598,Hannibal Mejbri,0.000000,0.000000,0.000000,0.055556,0.666667,0.000000,0.000000,25.000000,1.315789,0.0,0.0,8.0
599,Tyrese Francois,0.000000,0.000000,0.555556,0.166667,0.666667,0.722222,0.000000,25.000000,46.536712,0.0,0.0,16.0


In [ ]:
final_data.sort_values(['goals_scored'],ascending=False)

,name,assists,goals_scored,goals_conceded,ict_index,influence,creativity,threat,value,minutes,bonus,clean_sheets,bps
40,Harry Kane,0.274507,0.660909,1.116588,9.610736,34.604265,16.851385,44.698062,113.161700,42.100390,0.0,1.0,11.0
216,Mohamed Salah,0.213624,0.575199,0.901479,10.471364,28.676991,22.257824,53.851569,119.391108,81.024342,0.0,0.0,16.0
47,Jamie Vardy,0.305834,0.506309,1.241908,7.437509,25.063543,11.215738,38.172415,97.798076,46.920531,1.0,0.0,8.0
88,Sergio Agüero,0.162603,0.485771,0.481423,7.068281,22.014272,10.876696,38.145638,110.112812,60.547016,1.0,0.0,4.0
251,Pierre-Emerick Aubameyang,0.128110,0.447043,0.984173,6.503439,21.177625,12.468931,31.495886,108.281698,48.930623,0.0,0.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,Jean-Philippe Gbamin,0.000000,0.000000,0.555556,0.598765,4.580247,1.277778,0.000000,40.123457,37.688326,0.0,0.0,21.0
379,José Ignacio Peleteiro Romallo,0.024691,0.000000,0.123457,0.427160,1.160494,1.404938,1.802469,14.469136,39.773739,0.0,0.0,12.0
380,Luke Freeman,0.000000,0.000000,0.112233,0.334456,1.037037,1.737374,0.561167,12.974186,50.590210,0.0,1.0,18.0
401,Frédéric Guilbert,0.019753,0.000000,0.355556,0.762469,4.456296,2.385185,0.760494,10.844444,1.304439,1.0,0.0,8.5


In [ ]:
np.random.choice(np.arange(1, 7), p=[0.1, 0.05, 0.05, 0.2, 0.4, 0.2])

1